<a href="https://colab.research.google.com/github/JamieCDev20/GAM705/blob/main/705Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

Imports

In [1]:
import numpy as np
!pip install pyquaternion
from pyquaternion import Quaternion

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


File Reader

In [175]:
class AnimFileReader():
  ## "#" breaks between nodes | ":" breaks between values

  def __init__(self, _path):
    self.path = _path
    self.animData = np.array(float)
    self.__ReadAnim__(self.path)

  def __ReadAnim__(self, path):
    file = open(path, 'r')
    overallData = file.read()
    overallData = overallData.split("SS")
    
    del overallData[0]

    jointCount = len(overallData)
    frameCount = len(overallData[0].split('#')[0].split(':')) - 1
    
    dataLayer = np.array(float)

    trainingData = np.array(float)

    for j in range(0, frameCount):
      for i in range(0, jointCount):

        rots = overallData[i].split('#')[0].split(':')
        poss = overallData[i].split('#')[1].split(':')

        del rots[0]
        del poss[0]

        #print(len(rots))
        #print(rots[0], rots[1], rots[2], rots[3])
        #print(dataLayer.shape)
        temp = rots[j].split(',')
        dataLayer = np.append(dataLayer, temp)
        
        temp = poss[j].split(',')
        dataLayer = np.append(dataLayer, temp)
        #print(dataLayer.shape)
      dataLayer = np.delete(dataLayer, 0)
      trainingData = np.append(trainingData, dataLayer);
      dataLayer = np.array(float)
      print("Completed: ", j + 1)

    print("Total its: ", (i + 1) * (j + 1))
    
    trainingData = np.delete(trainingData, 0)
    
    print(trainingData.shape)
    print(trainingData[0])
    
    self.animData = trainingData;
   

In [176]:
fr = AnimFileReader("SHO_EXORCISMWALK_1_UNCLEAN.txt")

Completed:  1
Completed:  2
Completed:  3
Completed:  4
Completed:  5
Completed:  6
Completed:  7
Completed:  8
Completed:  9
Completed:  10
Completed:  11
Completed:  12
Completed:  13
Completed:  14
Completed:  15
Completed:  16
Completed:  17
Completed:  18
Completed:  19
Completed:  20
Completed:  21
Completed:  22
Completed:  23
Completed:  24
Completed:  25
Completed:  26
Completed:  27
Completed:  28
Completed:  29
Completed:  30
Completed:  31
Completed:  32
Completed:  33
Completed:  34
Completed:  35
Completed:  36
Completed:  37
Completed:  38
Completed:  39
Completed:  40
Completed:  41
Completed:  42
Completed:  43
Completed:  44
Completed:  45
Completed:  46
Completed:  47
Completed:  48
Completed:  49
Completed:  50
Completed:  51
Completed:  52
Completed:  53
Completed:  54
Completed:  55
Completed:  56
Completed:  57
Completed:  58
Completed:  59
Completed:  60
Completed:  61
Completed:  62
Completed:  63
Completed:  64
Completed:  65
Completed:  66
Completed:  67
Comp

In [133]:
data = fr.animData
print(fr.animData.shape)
print(data.shape)
data = np.reshape(data, [221, len(data)/221])
print(fr.animData.shape)
print(data.shape)

(8205510,)
(8205510,)


TypeError: ignored

Testing

In [6]:
a = np.array([1, 2, 3, 4, 5], float)
b = np.array([6, 6, 6, 6, 6], float)
c = b-a
b = np.append(b, c);

print(b)


[6. 6. 6. 6. 6. 5. 4. 3. 2. 1.]
